In [70]:
context = {
	'weights': {

	},
	'WELL_COLUMN': 9
}

In [71]:
import random
import cv2
import numpy as np
from PIL import Image
import copy
from time import sleep

# Tetris game class
class Tetris:

    '''Tetris game class'''

    # BOARD
    MAP_EMPTY = 0
    MAP_BLOCK = 1
    MAP_PLAYER = 2
    BOARD_WIDTH = 10
    BOARD_HEIGHT = 20

    TETROMINOS = {
        0: { # I
            0: [(0,0), (1,0), (2,0), (3,0)],
            90: [(1,0), (1,1), (1,2), (1,3)],
            180: [(3,0), (2,0), (1,0), (0,0)],
            270: [(1,3), (1,2), (1,1), (1,0)],
        },
        1: { # T
            0: [(1,0), (0,1), (1,1), (2,1)],
            90: [(0,1), (1,2), (1,1), (1,0)],
            180: [(1,2), (2,1), (1,1), (0,1)],
            270: [(2,1), (1,0), (1,1), (1,2)],
        },
        2: { # L
            0: [(1,0), (1,1), (1,2), (2,2)],
            90: [(0,1), (1,1), (2,1), (2,0)],
            180: [(1,2), (1,1), (1,0), (0,0)],
            270: [(2,1), (1,1), (0,1), (0,2)],
        },
        3: { # J
            0: [(1,0), (1,1), (1,2), (0,2)],
            90: [(0,1), (1,1), (2,1), (2,2)],
            180: [(1,2), (1,1), (1,0), (2,0)],
            270: [(2,1), (1,1), (0,1), (0,0)],
        },
        4: { # Z
            0: [(0,0), (1,0), (1,1), (2,1)],
            90: [(0,2), (0,1), (1,1), (1,0)],
            180: [(2,1), (1,1), (1,0), (0,0)],
            270: [(1,0), (1,1), (0,1), (0,2)],
        },
        5: { # S
            0: [(2,0), (1,0), (1,1), (0,1)],
            90: [(0,0), (0,1), (1,1), (1,2)],
            180: [(0,1), (1,1), (1,0), (2,0)],
            270: [(1,2), (1,1), (0,1), (0,0)],
        },
        6: { # O
            0: [(1,0), (2,0), (1,1), (2,1)],
            90: [(1,0), (2,0), (1,1), (2,1)],
            180: [(1,0), (2,0), (1,1), (2,1)],
            270: [(1,0), (2,0), (1,1), (2,1)],
        }
    }

    COLORS = {
        0: (255, 255, 255),
        1: (247, 64, 99),
        2: (0, 167, 247),
    }


    def __init__(self, context):
        self.reset()
        self.context = context
        self.quit = False

    
    def reset(self):
        '''Resets the game, returning the current state'''
        self.board = [[0] * Tetris.BOARD_WIDTH for _ in range(Tetris.BOARD_HEIGHT)]
        self.game_over = False
        self.bag = list(range(len(Tetris.TETROMINOS)))
        random.shuffle(self.bag)
        self.next_piece = self.bag.pop()
        self._new_round()
        self.score = 0
        self.surfaceArray = [0] * Tetris.BOARD_WIDTH
        return self._get_board_props(self.board)


    def _get_rotated_piece(self):
        '''Returns the current piece, including rotation'''
        return Tetris.TETROMINOS[self.current_piece][self.current_rotation]


    def _get_complete_board(self):
        '''Returns the complete board, including the current piece'''
        piece = self._get_rotated_piece()
        piece = [np.add(x, self.current_pos) for x in piece]
        board = [x[:] for x in self.board]
        for x, y in piece:
            board[y][x] = Tetris.MAP_PLAYER
        return board


    def get_game_score(self):
        '''Returns the current game score.

        Each block placed counts as one.
        For lines cleared, it is used BOARD_WIDTH * lines_cleared ^ 2.
        '''
        return self.score
    
    def _new_round(self):
        '''Starts a new round (new piece)'''
        # Generate new bag with the pieces
        if len(self.bag) == 0:
            self.bag = list(range(len(Tetris.TETROMINOS)))
            random.shuffle(self.bag)
        
        self.current_piece = self.next_piece
        self.next_piece = self.bag.pop()
        self.current_pos = [3, 0]
        self.current_rotation = 0

        if self._check_collision(self._get_rotated_piece(), self.current_pos):
            self.game_over = True


    def _check_collision(self, piece, pos):
        '''Check if there is a collision between the current piece and the board'''
        for x, y in piece:
            x += pos[0]
            y += pos[1]
            if x < 0 or x >= Tetris.BOARD_WIDTH \
                    or y < 0 or y >= Tetris.BOARD_HEIGHT \
                    or self.board[y][x] == Tetris.MAP_BLOCK:
                return True
        return False


    def _rotate(self, angle):
        '''Change the current rotation'''
        r = self.current_rotation + angle

        if r == 360:
            r = 0
        if r < 0:
            r += 360
        elif r > 360:
            r -= 360

        self.current_rotation = r


    def _add_piece_to_board(self, piece, pos):
        '''Place a piece in the board, returning the resulting board'''        
        board = [x[:] for x in self.board]
        for x, y in piece:
            board[y + pos[1]][x + pos[0]] = Tetris.MAP_BLOCK
        return board


    def _clear_lines(self, board):
        '''Clears completed lines in a board'''
        # Check if lines can be cleared
        lines_to_clear = [index for index, row in enumerate(board) if sum(row) == Tetris.BOARD_WIDTH]
        if lines_to_clear:
            board = [row for index, row in enumerate(board) if index not in lines_to_clear]
            # Add new lines at the top
            for _ in lines_to_clear:
                board.insert(0, [0 for _ in range(Tetris.BOARD_WIDTH)])
        return len(lines_to_clear), board

    def _number_of_holes(self, board):
        '''Number of holes in the board (empty sqquare with at least one block above it)'''
        holes = 0

        for col in zip(*board):
            i = 0
            while i < Tetris.BOARD_HEIGHT and col[i] != Tetris.MAP_BLOCK:
                i += 1
            holes += len([x for x in col[i+1:] if x == Tetris.MAP_EMPTY])
        return holes


    def _bumpiness(self, board):
        '''Sum of the differences of heights between pair of columns'''
        total_bumpiness = 0
        max_bumpiness = 0
        min_ys = []

        for col in zip(*board):
            i = 0
            while i < Tetris.BOARD_HEIGHT and col[i] != Tetris.MAP_BLOCK:
                i += 1
            min_ys.append(i)
        
        for i in range(len(min_ys) - 1):
            bumpiness = abs(min_ys[i] - min_ys[i+1])
            max_bumpiness = max(bumpiness, max_bumpiness)
            total_bumpiness += abs(min_ys[i] - min_ys[i+1])

        return total_bumpiness, max_bumpiness


    def _height(self, board):
        '''Sum and maximum height of the board'''
        sum_height = 0
        max_height = 0
        min_height = Tetris.BOARD_HEIGHT

        for col in zip(*board):
            i = 0
            while i < Tetris.BOARD_HEIGHT and col[i] == Tetris.MAP_EMPTY:
                i += 1
            height = Tetris.BOARD_HEIGHT - i
            sum_height += height
            if height > max_height:
                max_height = height
            elif height < min_height:
                min_height = height

        return sum_height, max_height, min_height


    def _get_board_props(self, board):
        '''Get properties of the board'''
        lines, board = self._clear_lines(board)
        holes = self._number_of_holes(board)
        total_bumpiness, max_bumpiness = self._bumpiness(board)
        sum_height, max_height, min_height = self._height(board)
        return [lines, holes, total_bumpiness, sum_height]


    def get_next_states(self):
        '''Get all possible next states'''
        states = {}
        piece_id = self.current_piece
        
        if piece_id == 6: 
            rotations = [0]
        elif piece_id == 0:
            rotations = [0, 90]
        else:
            rotations = [0, 90, 180, 270]

        # For all rotations
        for rotation in rotations:
            piece = Tetris.TETROMINOS[piece_id][rotation]
            min_x = min([p[0] for p in piece])
            max_x = max([p[0] for p in piece])

            # For all positions
            for x in range(-min_x, Tetris.BOARD_WIDTH - max_x):
                pos = [x, 0]

                # Drop piece
                while not self._check_collision(piece, pos):
                    pos[1] += 1
                pos[1] -= 1

                # Valid move
                if pos[1] >= 0:
                    board = self._add_piece_to_board(piece, pos)
                    states[(x, rotation)] = self._get_board_props(board)

        return states

    def get_next(self):
        possible = {}
        piece_id = self.current_piece
        
        if piece_id == 6: 
            rotations = [0]
        elif piece_id == 0:
            rotations = [0, 90]
        else:
            rotations = [0, 90, 180, 270]

        # For all rotations
        for rotation in rotations:
            piece = Tetris.TETROMINOS[piece_id][rotation]
            min_x = min([p[0] for p in piece])
            max_x = max([p[0] for p in piece])

            # For all positions
            for x in range(-min_x, Tetris.BOARD_WIDTH - max_x):
                pos = [x, 0]

                # Drop piece
                while not self._check_collision(piece, pos):
                    pos[1] += 1
                pos[1] -= 1

                # Valid move
                if pos[1] >= 0:
                    board = self._add_piece_to_board(piece, pos)
                    line_clears, board = self._clear_lines(board)

                    possible[(x, rotation)] = {'board': board, 'lines': line_clears}

        return possible


    def get_state_size(self):
        '''Size of the state'''
        return 4


    def play(self, x, rotation, render=False, render_delay=None):
        '''Makes a play given a position and a rotation, returning the reward and if the game is over'''
        self.current_pos = [x, 0]
        self.current_rotation = rotation

        # Drop piece
        while not self._check_collision(self._get_rotated_piece(), self.current_pos):
            if render:
                self.render()
                if render_delay:
                    sleep(render_delay)
            self.current_pos[1] += 1
        self.current_pos[1] -= 1

        # Update board and calculate score        
        self.board = self._add_piece_to_board(self._get_rotated_piece(), self.current_pos)
        lines_cleared, self.board = self._clear_lines(self.board)
        score = 1 + (lines_cleared ** 2) * Tetris.BOARD_WIDTH
        self.score += score

        # Start new round
        self._new_round()
        if self.game_over:
            score -= 2

        return score, self.game_over


    def render(self):
        '''Renders the current board'''
        img = [Tetris.COLORS[p] for row in self._get_complete_board() for p in row]
        img = np.array(img).reshape(Tetris.BOARD_HEIGHT, Tetris.BOARD_WIDTH, 3).astype(np.uint8)
        img = img[..., ::-1] # Convert RRG to BGR (used by cv2)
        img = Image.fromarray(img, 'RGB')
        img = img.resize((Tetris.BOARD_WIDTH * 25, Tetris.BOARD_HEIGHT * 25))
        img = np.array(img)
        cv2.putText(img, str(self.score), (22, 22), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
        cv2.imshow('image', np.array(img))
        key = cv2.waitKey(1)
        if (key % 256 == ord('q')):
            self.quit = True
            cv2.destroyAllWindows()

In [72]:
### UTILS

FULL_ROW = np.uint32(1023)
HOLE_WEIGHT_BIT = np.uint(1) << 30

def TUCK_SETUP_BIT(x):
	return np.uint(1) << (29 - x)

def HOLE_BIT(x):
	return np.uint(1) << (19 - x)

ALL_TUCK_SETUP_BITS = np.uint(1023) << 20

ALL_HOLE_BITS = np.uint(1023) << 10

ALL_AUXILIARY_BITS = ~np.uint(1023)

TUCK_SETUP_HOLE_PROPORTION = 0.81

FLOAT_EPSILON = 0.000001

In [73]:
from bitstring import BitArray
import math

context = {
	'WELL_COLUMN': 9,
	'USE_RIGHT_WELL_FEATURES': 1,
	'scareHeight': 3,
	'maxSafeCol9': 5,
	'weights': {
		'avgHeightCoef': -7,
		'builtOutLeftCoef': 2,
		'burnCoef': -14,
		'coveredWellCoef': -5,
		'col9Coef': -3,
		'deathCoef': -3000,
		'extremeGapCoef': -3,
		'holeCoef': -50,
		'holeWeightCoef': 0,
		'inaccessibleLeftCoef': -100,
		'inaccessiblerightCoef': -200,
		'tetrisCoef': 40,
		'tetrisReadyCoef': 6,
		'surfaceCoef': 1,
		'surfaceLeft': 0,
		'unableToBurnCoef': -0.5
	}
}

class Evaluator():
	def __init__(self, context):
		self.board = [0] * 20
		self.surfaceArray = [0] * 10
		self.context = context

	def update(self, board):
		for i, arr_row in enumerate(board):
			arr_row = [str(1) if i > 0 else str(0) for i in arr_row]
			row = np.uint32(BitArray(bin="".join(arr_row)).int)
			self.board[i] = row

	def updSurfaceArray(self):
		for col in range(10):
			row = 0
			mask = 1 << (9 - col)
			while (row < 20 and not (self.board[row] & mask)):
				row += 1

			self.surfaceArray[col] = 20 - row

	def _rateSurface(self):
		score = 30
		for i in range(0, 9):
			if i == self.context['WELL_COLUMN'] or i + 1 == self.context['WELL_COLUMN']:
				continue
			diff = self.surfaceArray[i + 1] - self.surfaceArray[i]

			if (self.context['USE_RIGHT_WELL_FEATURES'] and i == 7 and self.context['WELL_COLUMN'] == 9 and diff < -2):
				diff = -2
			
			if (diff != 0):
				score -= math.abs(diff) ** 1.5

			if (diff >= 3 and (i == 0 or self.surfaceArray[i - 1] + self.surfaceArray[i] >= 3)):
				score -= 25

		return score
	
	def _getAverageHeight(self):
		avgHeight = 0
		weight = 0.1 if self.context['WELL_COLUMN'] >= 0 else 0.111111
		for i in range(10):
			if (i == self.context['WELL_COLUMN']):
				continue

			avgHeight += self.surfaceArray[i] * weight
		return avgHeight

	def _getAverageHeightFactor(self, avgHeight, scareHeight):
		diff = max(0, avgHeight - scareHeight)
		return diff ** 2

	def _getBuiltOutLeftFactor(self, avgHeight, scareHeight):
		if (not self.context['USE_RIGHT_WELL_FEATURES']):
			return 0

		heightRatio = max(1, avgHeight) / max(2, scareHeight)
		heightDiff = 0.5 * (self.surfaceArray[0] - avgHeight) + 0.5 * (self.surfaceArray[0] - self.surfaceArray[1])

		if heightDiff < 0:
			softenedHeightRatio = 0.5 * (heightRatio + 1)
			return -0.5 * heightDiff * heightDiff * softenedHeightRatio

		r = 21 - self.surfaceArray[0]
		while r < 20:
			if (not (self.board[r] & (1 << 9))):
				return 0
			r += 1

		return heightRatio * heightDiff

	def _getLeftSurfaceFactor(self, max5TapHeight):
		max5TapHeight = max(0, max5TapHeight)
		for r in range(20 - self.surfaceArray[0], 20):
			if (self.board[r] & HOLE_BIT(0)):
				return -5

		if (self.surfaceArray[1] > max5TapHeight and self.surfaceArray[1] > self.surfaceArray[0]):
			return self.surfaceArray[0] - self.surfaceArray[1]
		return 0

	def _getCol9Factor(self, col9Height, maxSafeCol9Height):
		if (not self.context['USE_RIGHT_WELL_FEATURES']):
			return 0

		if (col9Height <= maxSafeCol9Height):
			return 0

		diff = col9Height - maxSafeCol9Height
		return diff * diff

	def _getCoveredWellFactor(self, scareHeight):
		if (self.context['WELL_COLUMN'] == -1):
			return 0

		mask = (1 << (9 - self.context['WELL_COLUMN']))

		for r in range(20):
			if (self.board[r] & mask):
				difficultyMultiplier = 10 if (self.board[r] & (ALL_HOLE_BITS | ALL_TUCK_SETUP_BITS)) > 0 else 1
				heightRatio = (20 - r) / max(3, scareHeight)
				return heightRatio * heightRatio * heightRatio * difficultyMultiplier

		return 0

	def _getGuaranteedBurnsFactor(self):
		if (self.context['WELL_COLUMN'] == -1):
			return 0
		
		wellMask = 1 << (9 - self.context['WELL_COLUMN'])
		guaranteedBurns = 0
		for r in range(20):
			if (self.board[r] & wellMask) or (self.board[r] & HOLE_WEIGHT_BIT):
				guaranteedBurns += 1
		
		return guaranteedBurns
		


	def _getHoleWeightFactor(self):
		if (self.context['WELL_COLUMN'] == -1):
			return 0
		
		holeWeight = 0
		for  r in range(20):
			if (self.board[r] & HOLE_WEIGHT_BIT):
				holeWeight += 1
		return holeWeight
	
	def _getLikelyBurnsFactor(self, maxSafeCol9):
		if (self.context['WELL_COLUMN'] != 9 or self.context['USE_RIGHT_WELL_FEATURES']):
			return 0

		col9 = self.surfaceArray[8]
		col8 = self.surfaceArray[7]

		lowestGoodColumn9 = min(maxSafeCol9, col8 - 2)
		if (col9 >= lowestGoodColumn9):
			return 0

		diff = lowestGoodColumn9 - col9
		return math.ceil(diff / 2) * 0.6
	
	def _getInaccessibleLeftFactor(self, maxAccessibleLeftSurface):

		highestRowOfCol1 = 19 - self.surfaceArray[0]
		needs5TapForDig = self.board[highestRowOfCol1] & HOLE_WEIGHT_BIT
		needs5TapForBurn = self.context['WELL_COLUMN'] == 9 and self.surfaceArray[0] <= self.surfaceArray[8]
		needs5Tap = needs5TapForDig or needs5TapForBurn

		hasHoleInLeft = False
		for r in range(highestRowOfCol1 + 1, highestRowOfCol1 + 4):
			if (self.board[r] & (HOLE_BIT(0) | HOLE_BIT(1) | HOLE_BIT(2))):
				hasHoleInLeft = True
				break

		if self.surfaceArray[0] > maxAccessibleLeftSurface[0] and self.surfaceArray[0] >= self.surfaceArray[1] and not needs5Tap and not hasHoleInLeft:
			return 0

		highestAbove = 0
		for i in range(7):
			if self.surfaceArray[i] > maxAccessibleLeftSurface[i]:
				highsetAbove = max(highestAbove, self.surfaceArray[i] - maxAccessibleLeftSurface[i])

		return 0 if highestAbove == 0 else (1.0 + 0.2 * highestAbove * highestAbove)

	def _getInaccessibleRightFactor(self, maxAccessibleRightSurface):
		needsRightTap = self.surfaceArray[9] < self.surfaceArray[8]
		if (self.surfaceArray[9] > maxAccessibleRightSurface[9] and not needsRightTap):
			return 0

		highestAbove = 0
		for i in range(5, 10):
			if (self.surfaceArray[i] > maxAccessibleRightSurface[i]):
				highestAbove = max(highestAbove, self.surfaceArray[i] - maxAccessibleRightSurface[i])

		return 0 if highestAbove == 0 else 1 + 0.2 * highestAbove * highestAbove

	def _getLineClearFactor(self, numLinesCleared, shouldRewardLineClears):
		return self.context['weights']['tetrisCoef'] if numLinesCleared == 4 else self.context['weights']['burnCoef'] * int(numLinesCleared) * (-0.25 if shouldRewardLineClears else 1)

	def _getUnableToBurnFactor(self, scareHeight):
		if (not self.context['USE_RIGHT_WELL_FEATURES']):
			return 0

		totalPenalty = 0

		col9Height = self.surfaceArray[8]

		if col9Height <= 0:
			return 0

		if col9Height <= self.surfaceArray[9]:
			while self.board[20 - col9Height] & 1:
				col9Height -= 1
		
		for c in range(8):
			if (self.surfaceArray[c] < col9Height):
				thisCol = self.surfaceArray[c]
				prevCol = 99 if c == 0 else self.surfaceArray[c - 1]
				nextCol = 99 if c == 9 else self.surfaceArray[c + 1]

				if prevCol - thisCol >= 3 and nextCol - thisCol >= 3:
					totalPenalty += 100
					break
				
				diff = col9Height - self.surfaceArray[c]
				totalPenalty += diff * diff

		col9Row = 20 - col9Height

		for r in range(max(0, col9Row - 1), col9Row + 1):
			if (self.board[r] & ALL_HOLE_BITS):
				totalPenalty += 50
		
		for r in range(max(0, col9Row - 2), min(19, col9Row + 2)):
			if (self.board[r] & ALL_HOLE_BITS):
				totalPenalty += 50

		scareRatio = col9Height / max(2, scareHeight)
		heightMultiplier = scareRatio * scareRatio * scareRatio

		return totalPenalty * heightMultiplier

	def _isTetrisReady(self):
		wellColHeight = self.surfaceArray[self.context['WELL_COLUMN']]
		if (wellColHeight > 16):
			return 0
		
		wellMask = (1 << (9 - self.context['WELL_COLUMN']))
		idealRowMask = FULL_ROW & ~wellMask

		for r in range(4):
			if (self.board[19 - wellColHeight - r] & FULL_ROW != idealRowMask):
				return False
		
		return True

	def _analyzeHole(self, r, c):
		board_copy = copy.deepcopy(self.board)
		if (
			(c >= 2 and ((board_copy[r] >> (9-c)) & 7) == 0) or
			(c <= 7 and ((board_copy[r] >> (7-c)) & 7) == 0)
		):
			board_copy |= TUCK_SETUP_BIT(c)
			return TUCK_SETUP_HOLE_PROPORTION

		board_copy[r] |= HOLE_BIT(c)
		return 1

		

	def _updateSurfaceAndHoles(self):
		board_copy = copy.deepcopy(self.board)
		surface_copy = copy.deepcopy(self.surfaceArray)
		for i in range(20):
			board_copy[i] &= ~ALL_AUXILIARY_BITS

		numHoles = 0
		for c in range(10):
			mask = 1 << (9 - c)
			r = 20 - self.surfaceArray[c]
			while r >= 0 and r < 20 and not (board_copy[r] & mask):
				r += 1

			surface_copy[c] = 20 - r

			r - max(0, r)
			r = min(19, r)
			lowestHoleInCol = -1
			while (r < 20):
				if (not (board_copy[r] & mask) and c != self.context['WELL_COLUMN']):
					rating = self._analyzeHole(r, c)
					if rating > TUCK_SETUP_HOLE_PROPORTION + FLOAT_EPSILON:
						lowestHoleInCol = r

					numHoles += rating
				r += 1
			for r in range(lowestHoleInCol - 1, 20 - self.surfaceArray[c] - 1, -1):
				board_copy[r] |= HOLE_WEIGHT_BIT
				
		return numHoles

	# def inaccessible():
	# 	maxAccessibleLeft5Surface = [0] * 10
	# 	maxAccessibleRight5Surface = [0] * 10
	# 	for i in range(10):
	# 		maxAccessibleLeft5Surface[i] = 20
	# 		maxAccessibleRight5Surface[i] = 20
		
	# 	PIECE_T
	# 	bottomSurface

	# 	for tapIndex in range(5):
	# 		for i in range(4):
	# 			if (bottomSurface[i] == NONE):
	# 				continue
	# 			II

	
	def fastEval(self, lines_cleared):

		w = self.context['weights']
		scareHeight = self.context['scareHeight']
		maxSafeCol9 = self.context['maxSafeCol9']
		shouldRewardLineClears = True # self.context['shouldRewardLineClears']
		adjustedNumHoles = self._updateSurfaceAndHoles()

		avgHeight = self._getAverageHeight()
		avgHeightFactor = w['avgHeightCoef'] * self._getAverageHeightFactor(avgHeight, scareHeight)
		builtOutLeftFactor = w['builtOutLeftCoef'] * self._getBuiltOutLeftFactor(avgHeight, scareHeight)
		coveredWellFactor = w['coveredWellCoef'] * self._getCoveredWellFactor(scareHeight)
		guaranteedBurnsFactor = w['burnCoef'] * self._getGuaranteedBurnsFactor()
		likelyBurnsFactor = w['burnCoef'] * self._getLikelyBurnsFactor(maxSafeCol9)
		highCol9Factor = w['col9Coef'] * self._getCol9Factor(self.surfaceArray[8], maxSafeCol9)
		holeFactor = w['holeCoef'] * adjustedNumHoles
		holeWeightFactor = w['holeWeightCoef'] * self._getHoleWeightFactor() if abs(w['holeWeightCoef']) > FLOAT_EPSILON else 0
		# inaccessibleLeftFactor = w['inaccessibleLeftCoef'] * self._getInaccessibleLeftFactor()
		# inaccessibleRightFactor = w['inaccessibleRightCoef'] * self._getInaccessibleRightFactor()
		inaccessibleLeftFactor = 0
		inaccessibleRightFactor = 0
		lineClearFactor = self._getLineClearFactor(lines_cleared, True)
		surfaceFactor = w['surfaceCoef'] * self._rateSurface()
		surfaceLeftFactor = 0 # w['surfaceLeftCoef'] * self._getLeftSurfaceFactor(what)

		tetrisReadyFactor = w['tetrisReadyCoef'] if (self.context['WELL_COLUMN'] >= 0 and self._isTetrisReady()) else 0
		unableToBurnFactor = w['unableToBurnCoef'] * self._getUnableToBurnFactor(scareHeight)

		total = surfaceFactor + surfaceLeftFactor + avgHeightFactor + lineClearFactor + holeFactor + holeWeightFactor + guaranteedBurnsFactor + likelyBurnsFactor + inaccessibleLeftFactor + inaccessibleRightFactor + coveredWellFactor + highCol9Factor + tetrisReadyFactor + builtOutLeftFactor + unableToBurnFactor
		return total

	
	# getAverageHeight
	# getAverageHeightFactor
	# getBuiltOutLeftFactor
	# getCoveredWellFactor
	# getGuaranteedBurnsFactor
	# getLikelyBurnsFazctor
	# getCol9Factor
	# getHoleWeightFactor
	# getInaccessibleLeftFactor
	# getInaccessibleRightFactor
	# getLineClearFactor
	# rateSurface
	# getLeftSurfaceFactor
	# isTetrisReady
	# getUnableToBurnFactor

In [82]:
eva = Evaluator(context)
env = Tetris(context)

max_steps = 10
steps = 0
render_delay = None
done = False

done = False
env.reset()
while not done and (not max_steps or steps < max_steps):
	next_states = env.get_next_states()

	possible = env.get_next()
	best_stuff = {}
	for action, state in possible.items():
		temp_board, temp_lines = state['board'], state['lines']
		eva.update(temp_board)
		best_stuff[action] = eva.fastEval(temp_lines)

	best_score = min(best_stuff.values())
	best_action = None
	for action, score in best_stuff.items():
		if score == best_score:
			best_action = action
			break

	print(best_action)
	reward, done = env.play(best_action[0], best_action[1], render=True,
							render_delay=render_delay)
	
	current_state = next_states[best_action]
	steps += 1

(8, 90)
(8, 180)
(8, 90)
(8, 90)
(8, 0)
(8, 90)
(6, 0)
(5, 0)
(3, 0)


: 

In [75]:
eva._getAverageHeight()
eva._getAverageHeightFactor(5, 10)
eva._getBuiltOutLeftFactor(5, 10)
eva._getCol9Factor(20, 10)
eva._getCoveredWellFactor(10)
eva._getGuaranteedBurnsFactor()
eva._getLeftSurfaceFactor(5)
eva._rateSurface()
eva._getHoleWeightFactor()
eva._getLikelyBurnsFactor(10)
eva._getInaccessibleLeftFactor([5, 2, 3, 3, 6, 5, 2 ,4 ,4, 5])

IndexError: list index out of range

In [ ]:
eva.updSurfaceArray()
eva.surfaceArray

[18, 8, 8, 7, 7, 3, 3, 0, 0, 0]

In [ ]:
from tetris import Tetris
from datetime import datetime
from statistics import mean, median
import random
import numpy as np

# Run dqn with Tetris
def dqn():
    env = Tetris()
    episodes = 2000
    max_steps = None
    epsilon_stop_episode = 1400
    mem_size = 20000
    discount = 0.95
    batch_size = 512
    epochs = 1
    render_every = 100
    log_every = 50
    replay_start_size = 2000
    train_every = 1
    n_neurons = [32, 32]
    render_delay = None
    activations = ['relu', 'relu', 'linear']

    agent = DQNAgent(env.get_state_size(),
                     n_neurons=n_neurons, activations=activations,
                     epsilon_stop_episode=epsilon_stop_episode, mem_size=mem_size,
                     discount=discount, replay_start_size=replay_start_size)

    # latest = tf.train.latest_checkpoint('training_2')

    # agent.model.load_weights(latest)

    # log_dir = f'logs/tetris-nn={str(n_neurons)}-mem={mem_size}-bs={batch_size}-e={epochs}-{datetime.now().strftime("%Y%m%d-%H%M%S")}'
    # log = CustomTensorBoard(log_dir=log_dir)

    scores = []

    for episode in tqdm(range(episodes)):
        current_state = env.reset()
        done = False
        steps = 0

        if render_every and episode % render_every == 0:
            render = True
        else:
            render = False

        # Game
        while not done and (not max_steps or steps < max_steps):
            next_states = env.get_next_states()
            best_state = agent.best_state(next_states.values())
            print("Best_state:",best_state)
            
            best_action = None
            for action, state in next_states.items():
                if state == best_state:
                    best_action = action
                    break

            reward, done = env.play(best_action[0], best_action[1], render=render,
                                    render_delay=render_delay)
            
            agent.add_to_memory(current_state, next_states[best_action], reward, done)
            current_state = next_states[best_action]
            steps += 1

        scores.append(env.get_game_score())

        if episode % 50 == 0:
            print('Score:',np.mean(scores[-log_every:]))

        # Train
        if episode % 100 == 0:
            agent.train(batch_size=batch_size, epochs=epochs, episode=0)
        elif episode % 50 == 0:
            agent.train(batch_size=batch_size, epochs=epochs, episode=episode)
        else:
            agent.train(batch_size=batch_size, epochs=epochs)

if __name__ == "__main__":
    dqn()